In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tensorflow import keras
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, BatchNormalization, Flatten, Dropout
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

## Read the data

In [ ]:
df_train = pd.read_csv("/kaggle/input/fashionmnist/fashion-mnist_train.csv")
df_train

In [ ]:
df_test = pd.read_csv("/kaggle/input/fashionmnist/fashion-mnist_test.csv")
df_test

## Vizualize the train and images

In [ ]:
import torch
import torchvision
import torch.nn as nn
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import numpy as np
from torchvision.datasets import FashionMNIST
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline


dataset = FashionMNIST(root='data/', download = True, train=True, transform = ToTensor())

In [ ]:
test_dataset = FashionMNIST(root='data/', train=False, transform = ToTensor())
len(test_dataset)

In [ ]:
image, label = dataset[4]
print('image.shape:', image.shape)
plt.imshow(image[0], cmap='gray')
print('Label:', label)

In [ ]:
val_size = 10000
train_size = len(dataset) - val_size

train_ds,val_ds = random_split(dataset,[train_size,val_size])
len(train_ds),len(val_ds)

In [ ]:
batch_size = 128
train_loader =DataLoader(train_ds,batch_size,shuffle=True,num_workers=4,pin_memory=True)
val_loader =DataLoader(val_ds,batch_size*2,num_workers=4,pin_memory=True)

In [ ]:
for images,_ in train_loader:
    print('images.shape:', images.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
    break

## Train and test split

In [ ]:
y = df_train.label.values.reshape(-1,1) #musím nejdřív spustit y
X = df_train.drop(['label'], axis=1).values

In [ ]:
X.shape

In [ ]:
X = X.reshape(-1, 28, 28, 1)

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Train and test data preprocessing for CNN

In [ ]:
y_train_categorical = tf.keras.utils.to_categorical(y_train, 10)
y_train_categorical[0]

In [ ]:
y_test_categorical = tf.keras.utils.to_categorical(y_test, 10)
y_train_categorical[0]

X_test = X_test.astype('float32')
X_test.shape

In [ ]:
test = X_test[0]
test.shape

## Model Training preprocessing for CNN

## First basic CNN made manually- model1

- with batchnorm, MaxPool, Flatten, Dense output 
- with augmentation 

In [ ]:
import numpy as np
import random as python_random

python_random.seed(123) #TOLE JE KOD, KTERÝ KDYŽ SPUSTÍM VÍCEKRÁT, VŽDY MÁM STEJNÉ VSTUPNÍ PODMÍNKY
np.random.seed(123)
tf.random.set_seed(1234)
import os
os.environ['PYTHONHASHSEED'] = '0' 

model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Conv2D(filters=4, kernel_size=(3,3), activation='relu', input_shape=(28, 28, 1) ) )
model1.add(tf.keras.layers.BatchNormalization())
model1.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3)))
model1.add(tf.keras.layers.BatchNormalization())
model1.add(tf.keras.layers.ReLU())

model1.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3)))
model1.add(tf.keras.layers.BatchNormalization())
model1.add(tf.keras.layers.ReLU())

model1.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3)))
model1.add(tf.keras.layers.BatchNormalization())
model1.add(tf.keras.layers.ReLU())

model1.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(10, activation='softmax' ))

model1.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['acc'])   
model1.summary()

In [ ]:
model1.fit(X_train, y_train_categorical,validation_data=(X_test, y_test_categorical), epochs=10)

## Second CNN with resiual joining manually (based on ResNet)- model2

- with residuals joining of layers, batchnorm, GlobalAveragePooling2D,  
- with augmentation 

In [ ]:
import numpy as np
import random as python_random

python_random.seed(123) #TOLE JE KOD, KTERÝ KDYŽ SPUSTÍM VÍCEKRÁT, VŽDY MÁM STEJNÉ VSTUPNÍ PODMÍNKY
np.random.seed(123)
tf.random.set_seed(1234)
import os
os.environ['PYTHONHASHSEED'] = '0'

input_layer = tf.keras.layers.Input(shape=(28,28,1))
x = input_layer #na vstupu mám tyto fotky
x = tf.keras.layers.Normalization(mean=X_train.mean(), variance=X_train.var())(x) #FUNKCIONÁLNÍ API, 

filters_l = [32,64,128,128] #pole počtu filtrů, vykonám loop tolikrát s různým nastavením filtrů

#první konvoluce, na inicialzaci kernelu používám "he_unform", které jsou vhodné, 
x = tf.keras.layers.Conv2D(filters=filters_l[0], kernel_size=(3,3), kernel_initializer='he_uniform', activation='elu', padding='same')(x)
for i, filters in enumerate(filters_l): #postupně přes pole filters vytvářím vrstvy
    if i != len(filters_l)-1:
        downsample = True
    else:
        downsample = False
    
    res_con1 = x  
    x = tf.keras.layers.BatchNormalization()(x) #batch normalizace
    x = tf.keras.layers.ELU()(x)  #elu  poté konvoluce
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3,3), kernel_initializer='he_uniform', padding='same' )(x)
    x = tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ELU()(x) 
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3,3), kernel_initializer='he_uniform', padding='same' )(x)
    x = tf.keras.layers.Add()([res_con1,x]) #reziduální spojení
    
    res_con2 = x  
    x = tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ELU()(x) 
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3,3), kernel_initializer='he_uniform', padding='same' )(x)
    x = tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ELU()(x) 
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3,3), kernel_initializer='he_uniform', padding='same' )(x)
    x = tf.keras.layers.Add()([res_con1,res_con2,x])
    
    res_con3 = x  
    x = tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ELU()(x) 
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3,3), kernel_initializer='he_uniform', padding='same' )(x)
    x = tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ELU()(x) 
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3,3), kernel_initializer='he_uniform', padding='same' )(x)
    x = tf.keras.layers.Add()([res_con1,res_con2,res_con3,x])
    
    res_con = x
    if downsample: #downsampling, když stride je 2, což mi výstup 2xzmenší u convoluce, padding-same značí, že pokud není stride, tak výstup kovnoluce je stejný jako vstup
        res_con = tf.keras.layers.BatchNormalization()(res_con)
        res_con = tf.keras.layers.ELU()(res_con) 
        res_con = tf.keras.layers.Conv2D(filters=(filters if not downsample else filters_l[i+1]), kernel_size=(3,3), kernel_initializer='he_uniform', strides=2, padding='same' )(res_con)
        
     
    x = tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ELU()(x) 
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=(3,3), kernel_initializer='he_uniform', padding='same' )(x)
    x = tf.keras.layers.BatchNormalization()(x) 
    x = tf.keras.layers.ELU()(x) 
    x = tf.keras.layers.Conv2D(filters=(filters if not downsample else filters_l[i+1]), kernel_size=(3,3), kernel_initializer='he_uniform', strides=(2 if downsample else 1), padding='same' )(x)
    x = tf.keras.layers.Add()([res_con,x])
    
x = tf.keras.layers.GlobalAveragePooling2D()(x) #nakonci poslední vsvy neudělám MaxPooling, ale GlobalAveraooling, což zlepší přesnost!!
x = tf.keras.layers.Dense(10, activation='softmax')(x)
model2 = tf.keras.Model(input_layer, x)
model2.compile(optimizer=tf.keras.optimizers.Adam(0.001), #leraning rate pro 1 by se to ani netrénovalo, pro malé číslo zase strašně dlouho
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=['acc'])   #labelsmooting místo too, aby všechny labely, které byly 0 mi předělalo na 0.1 a labely 1 předělalo na 0.9 a to způsobí, že se to nebude tolikrát přetrénovávat

model2.summary()

In [ ]:
model2.fit(X_train, y_train_categorical,validation_data=(X_test, y_test_categorical), epochs=10)

## Third CNN with VGG16(transfer learning)- model3

- Note: we cannot use EfficientNetB0, because the minimum iput shape must be (32,32) but we have only (28,28) and do not solve this problem with any rescaling methods

In [ ]:
"""
import numpy as np
import random as python_random

python_random.seed(123) #TOLE JE KOD, KTERÝ KDYŽ SPUSTÍM VÍCEKRÁT, VŽDY MÁM STEJNÉ VSTUPNÍ PODMÍNKY
np.random.seed(123)
tf.random.set_seed(1234)
import os
os.environ['PYTHONHASHSEED'] = '0'


model3 = tf.keras.applications.VGG16(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(28,28,1),
    pooling=None,
    classes=10,
    classifier_activation="softmax"
)
model3.summary()
model3.compile(optimizer='adam', loss='caterical_crossentropy')
"""


In [ ]:
#model3.fit(X_train, y_train_categorical,validation_data=(X_test, y_test_categorical), epochs=10)

## Augmentation of the CNN

- no augmentation, because we do not have the pictures in input data set

In [ ]:
"""
train_datagen = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             zca_whitening=False,
                             rotation_range=10,
                             zoom_range=0.1,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False,
                             rescale=1./255)
valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(X_train, shuffle=True, target_size=(28,28),batch_size=128)
valid_generator = valid_datagen.flow_from_directory(X_test, shuffle=True, target_size=(28,28),batch_size=128)

"""

In [ ]:
"""

test_datagen = ImageDataGenerator(rescale=1./1.)

test_gen = test_datagen.flow_from_directory(directory='/kaggle/temp/test',target_size=(32,32),batch_size=64,class_mode=None,shuffle=False)

"""

In [ ]:
"""

test_gen.reset()
predictions_vecs = model.predict(test_gen)

predictions_final = np.argmax(predictions_vecs, axis=1)

"""

In [ ]:
model1.save('my_model1')
model2.save('my_model2')
#model3.save('my_model3')

## Prediction

In [ ]:
model1.predict(X_train[:2])
model2.predict(X_train[:2])
#model3.predict(x_train_reshaped[:2])